In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from transformers import ViTForImageClassification, ViTFeatureExtractor
import timm  # Para cargar EfficientNet y otros modelos de timm

In [ ]:
# Configuración de hiperparámetros
batch_size = 32
num_epochs = 20
learning_rate = 0.001

# Transformaciones de las imágenes (normalización CIFAR-100)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ajustamos para todos los modelos
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  # Normalización CIFAR-100
])

# Cargar el dataset CIFAR-100
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Cargar los modelos preentrenados
resnet = models.resnet50(pretrained=True)
efficientnet = timm.create_model('efficientnet_b0', pretrained=True, num_classes=100)
vit = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=100)

# Ajustar la capa de salida de los modelos
resnet.fc = nn.Linear(resnet.fc.in_features, 100)
efficientnet.classifier = nn.Linear(efficientnet.classifier.in_features, 100)
vit.classifier = nn.Linear(vit.classifier.in_features, 100)

# Mover los modelos a la GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet = resnet.to(device)
efficientnet = efficientnet.to(device)
vit = vit.to(device)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer_resnet = optim.Adam(resnet.parameters(), lr=learning_rate)
optimizer_efficientnet = optim.Adam(efficientnet.parameters(), lr=learning_rate)
optimizer_vit = optim.Adam(vit.parameters(), lr=learning_rate)

# Función para entrenar un modelo
def train_model(model, optimizer, train_loader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')
    return model

In [ ]:
print("Entrenando ResNet...")
resnet = train_model(resnet, optimizer_resnet, train_loader, num_epochs)
# torch.save(resnet, '/content/drive/MyDrive/Universidad/APIs y Desarrollo de inteligencia artificial en Python/Corte 2/Parcial_2_1/Punto_3/Models/resnet50_model.pkl')
torch.save(resnet, '/kaggle/working/resnet50_model.pkl')

In [ ]:
print("Entrenando EfficientNet...")
efficientnet = train_model(efficientnet, optimizer_efficientnet, train_loader, num_epochs)
# torch.save(efficientnet, '/content/drive/MyDrive/Universidad/APIs y Desarrollo de inteligencia artificial en Python/Corte 2/Parcial_2_1/Punto_3/Models/efficientnet_b0_model.pkl')
torch.save(resnet, '/kaggle/working/efficientnet_b0_model.pkl')